# Comment

Датасет http://www.cs.cmu.edu/~enron/
Уже скаченный и лежит в папке '/mnt/data/molchanov/datasets/maildir'

! pip install mail-parser

StreamingGraph -  словарь графов (1 граф соответствует 1 дню) ключ - дата (день)
Каждый граф - объект типа Graph, который хранит вершины и ребра (Edge) в виде списка смежности, 
что представляет из себя словарь словарей, ключи которых - имена нод графов в данном случае емэил адреса отправителя 
и получающего, например Graph['sender1@mail.ru']['reciver2@mail.ru'] это ребро, объект класса Edge - распарсенные письма
между 'sender1@mail.ru' и 'reciver2@mail.ru' в течении дня. Edge - Хранит в себе словарь свойств, на данный момент там есть число 
писем и Subject письма.

In [100]:
import time
import os
from os import listdir
from os import path as osp
import tqdm
import re
from collections import defaultdict
import numpy as np
from copy import copy, deepcopy
import datetime
import mailparser
from multiprocessing import Pool

base_path = '/mnt/data/molchanov/datasets/maildir'

def jn(p1, p2):
    return osp.join(p1, p2)

def jnb(p):
    return jn(base_path, p)

In [102]:
class Mapping:
    
    def __init__(self):
        self.__dict__ = dict()

    def __setitem__(self, key, item):
        self.__dict__[key] = item

    def __getitem__(self, key):
        return self.__dict__[key]

    def __repr__(self):
        return repr(self.__dict__)

    def __len__(self):
        return len(self.__dict__)

    def __delitem__(self, key):
        del self.__dict__[key]

    def clear(self):
        return self.__dict__.clear()

    def copy(self):
        return self.__dict__.copy()

    def has_key(self, k):
        return k in self.__dict__

    def update(self, *args, **kwargs):
        return self.__dict__.update(*args, **kwargs)

    def keys(self):
        return self.__dict__.keys()

    def values(self):
        return self.__dict__.values()

    def items(self):
        return self.__dict__.items()

    def pop(self, *args):
        return self.__dict__.pop(*args)

    def __cmp__(self, dict_):
        return self.__cmp__(self.__dict__, dict_)

    def __contains__(self, item):
        return item in self.__dict__

    def __iter__(self):
        return iter(self.__dict__)

    def __unicode__(self):
        return unicode(repr(self.__dict__))

# Stream Graph implementation

In [107]:
class Edge(Mapping):
    
    def __init__(self, d={}):
        super().__init__()
        self.__dict__ = d
        
    def __add__(self, other):
        cur = deepcopy(self)
        for key, val in other.items():
            if key in cur:
                cur[key] += deepcopy(val)
            else:
                cur[key] = deepcopy(val)
        return cur
    
    def __repr__(self):
        return str(self.__dict__)
    
def ddict():
    return defaultdict(Edge)
    
class Graph(Mapping):
    
    def __init__(self):
        #self.__dict__ = defaultdict(lambda : defaultdict(Edge))
        self.__dict__ = defaultdict(ddict)
    
    # srs - sourse mail addres
    # dst - destination mail addres
    # features - dict of key value features
    def add_pair(self, srs, dst, features):
        self[srs][dst] += Edge(features)
    
    def __repr__(self):
        return str(self.__dict__)
        

class StreamingGraph(Mapping):
    
    def __init__(self):
        self.__dict__ = defaultdict(Graph)
        
    # date - mail date
    # srs - sourse mail addres
    # dst - destination mail addres
    # features - dict of key value features
    def add_pair(self, date, srs, dst, features):
        self[date].add_pair(srs, dst, features)
        
    def __repr__(self):
        return str(self.__dict__)
    
    
def merge_streamin_graphs(lst_graphs):
    merged_graph = StreamingGraph()
    
    for stream_graph in tqdm.tqdm(lst_graphs):
        for day, day_graph in stream_graph.items():
            for srs, dst_dict in day_graph.items():
                for dst, dst_mail in dst_dict.items():
                    merged_graph.add_pair(day, srs, dst, dst_mail.__dict__)
                    
    res = {key: merged_graph[key] for key in sorted(merged_graph.keys())}
    return res

In [117]:
def parce_mail(pth):
    mail = mailparser.parse_from_file(pth)
    
    d = {}
    dt = mail.Date[:-6]
    mask = '%a, %d %b %Y %H:%M:%S %z'
    try:
        d['Date'] = datetime.datetime.strptime(dt, mask)
    except:
        mask = '%a, %d %b %Y %H:%M:%S'
        try:
            d['Date'] = datetime.datetime.strptime(dt, mask)
        except Exception as e:
            print('ERROR')
            print(pth)
            print(e)
            print('----------------------------------------------------------------')
            return None
    
    d['From'] = [m.lower() for _, m in mail.From]
    d['To'] = [m.lower() for _, m in mail.To]
    d['CC'] = [m.lower() for _, m in mail.CC]
    d['BCC'] = [m.lower() for _, m in mail.BCC]
    d['Subject'] = mail.Subject.lower()
    #d['Body'] = re.sub(r'\s', ' ', mail.body.lower())
    return d


def parce_mail_dict(streaming_graph, mail_dict):
    if mail_dict is None:
        return
    
    edges_types = {'To': 2, 'CC': 3, 'BCC': 4}
    
    date = mail_dict['Date']
    date_key = datetime.datetime(date.year, date.month, date.day)
    
    source = mail_dict['From'][0]
    subject = [mail_dict['Subject']]
    
    for dst_type in ['To', 'CC', 'BCC']:
        for dst in mail_dict[dst_type]:
            edge_features = {
                'edge_weight': 1,
                'edge_type': edges_types[dst_type],
                'Subject': subject,
                'isre': 'Re:' in subject
            }
            streaming_graph.add_pair(date_key, source, dst, edge_features)

    return streaming_graph


In [118]:
def scan_mails(streaming_graph, pth):
    if streaming_graph is None:
        streaming_graph = StreamingGraph()
        
    if osp.isdir(pth):
        for sub_pth in listdir(pth):
            streaming_graph = scan_mails(streaming_graph, jn(pth, sub_pth))
        return streaming_graph
    else:
        streaming_graph = parce_mail_dict(streaming_graph, parce_mail(pth))
        return streaming_graph
        
def scan_task(pth):
    return scan_mails(None, pth[0])
    

def build_graph():
    graphs = []
    with Pool(processes=32) as p:
        pathes = [(jn(base_path, employ), i) for i, employ in enumerate(listdir(base_path))]
        with tqdm.tqdm(total=len(pathes)) as pbar:
            for i, stream_graph in enumerate(p.imap_unordered(scan_task, pathes)):
                graphs.append(stream_graph)
                pbar.update()
    return merge_streamin_graphs(graphs)

# Построение графа

In [119]:
%%time
STREAMING_GRAPH = build_graph()

 71%|███████▏  | 107/150 [00:31<00:16,  2.54it/s]

ERROR
/mnt/data/molchanov/datasets/maildir/lokey-t/calendar/33.
strptime() argument 1 must be str, not list
----------------------------------------------------------------


100%|██████████| 150/150 [00:38<00:00,  3.86it/s]


CPU times: user 56.1 s, sys: 10.4 s, total: 1min 6s
Wall time: 2min 39s


# Обход графа (не полный)

In [120]:
for date, graph in STREAMING_GRAPH.items():
    print(f'DATE: {date}\n')
    for sender, recivers_dict in graph.items():
        print(f'    FROM: {sender}')
        for reciver, sender_revicer_mail in recivers_dict.items():
            print(f'        TO: {reciver}')
            for key, val in sender_revicer_mail.items():
                print(f'        {key}: {val}')
            break
        break
    print('\n\n___________________________________________________________________________________\n')
    break
    

DATE: 0001-05-30 00:00:00

    FROM: payables.ibuyit@enron.com
        TO: jeff.skilling@enron.com
        edge_weight: 2
        edge_type: 4
        Subject: ['action requested:  invoice requires coding/issue\n resolution/approval 00110100059665', 'action requested:  invoice requires coding/issue\n resolution/approval 00110100059665']
        isre: 0


___________________________________________________________________________________

